from sympy import *
s, t = symbols('s t')
f = (abs(s - 0.5) + 1)/(s-t+2)
r = integrate(f, (s, 0, 3))
simplify(r)

In [ ]:
import numpy as np
from math import *
import matplotlib.pyplot as plt

In [ ]:
def K(t, s):
    return 1 / (s - t + 2)

def qe(s):
    return abs(s - 0.5) + 1

def f(t):
    return 3.5*(0.285714285714286*t-1.0)*log(2.0-t)-3.5*(0.285714285714286*t-1.0)*log(2.5-t)-1.5*(0.666666666666667*t-1.0)*log(2.5-t)+1.5*(0.666666666666667*t-1.0)*log(5.0-t)+2.0

ls = 3
lt = 1

Ns = [int(pow(2,i)) for i in range(2,11)]

def _A(N):
    hx = ls / N
    A = np.zeros((N+1, N+1))
    for j in range(N+1):
        tj = 0 + j*(lt/N)
        for i in range(1,N):
            si = 0 + i*(ls/N)
            A[j,i] = hx * K(tj, si)
        A[j,0] = 0.5 * hx * K(tj, 0)
        A[j,N] = 0.5 * hx * K(tj, ls)
    return A

def _b(N):
    b = np.zeros((N+1,1))
    for j in range(N+1):
        tj = 0 + j*(lt/N)
        b[j,0] =  f(tj)
    return b

def svd(A, b):
    U, S, V = np.linalg.svd(A, full_matrices=True)
    
    UTb = np.transpose(U) @ b 
    S1UTb = UTb
    for i in range(len(S1UTb)):
        if(S[i] != 0):
            S1UTb[i] = S1UTb[i] / S[i]
        else:
            S1UTb[i] = 0
    return np.transpose(V) @ S1UTb

def csvd(A, b, delta):
    U, S, V = np.linalg.svd(A, full_matrices=True)

    UTb = np.transpose(U) @ b 
    S1UTb = UTb
    for i in range(len(S1UTb)):
        if(S[i] > delta):
            S1UTb[i] = S1UTb[i] / S[i]
        else:
            S1UTb[i] = 0
    return np.transpose(V) @ S1UTb

# 1

In [ ]:
x = np.linspace(0,ls, 1000)
y = [qe(p) for p in x]

bAdd = [(qe(0) * (1000 - i) + i * qe(ls)) / 1000 for i in range(1000) ]
    
plt.plot(x, y, label = "Точное решение")
#plt.plot(x, bAdd, label = "Априорная информация")
plt.legend()
plt.xlabel('s')
plt.ylabel('q(s)')
plt.grid(True)
plt.show()

In [ ]:
x = np.linspace(0,lt, 1000)
y = [f(p) for p in x]

plt.plot(x, y, label = "Правая часть f(t)")
plt.legend()
plt.xlabel('t')
plt.ylabel('f(t)')
plt.grid(True)
plt.show()

In [ ]:
x = np.linspace(0,ls, 1000)
y = [qe(p) for p in x]

for n in Ns:
    A = _A(n)
    b = _b(n)
    qsol = svd(A,b)
    
    plt.plot([0 + i * ls/n for i in range(0, n+1)], qsol, label = "Приближенное решение")
    plt.plot(x, y, label = "Точное решение")
    
    plt.legend()
    plt.title("График приближенного решения при N = " + str(n))
    plt.xlabel('s')
    plt.ylabel('q(s)')
    plt.grid(True)
    plt.show()

In [ ]:
Er = []
Er1 = []
for n in Ns:
    A = _A(n)
    b = _b(n)
    
    qexs = np.zeros((n+1,1))
    for i in range(n+1):
        qexs[i,0] =  qe(0 + i*(ls/n))

    qsol = svd(A,b)
    
    Er.append((np.transpose(A@qexs - b) @ (A@qexs - b))[0,0] / n)
    Er1.append((np.transpose(A@qsol - b) @ (A@qsol - b))[0,0] / n)

# MSE
plt.plot(Ns, np.log10(Er), label = "MSE точного решения")
plt.plot(Ns, np.log10(Er1), label = "MSE полученного решения")
plt.legend()
plt.xlabel('N')
plt.ylabel('log MSE')
plt.grid(True)
plt.show()

# 2

In [ ]:
S1 = []
S2 = []
for n in Ns:
    A = _A(n)
    u, s, vh = np.linalg.svd(A, full_matrices=True)
    S1.append(s[0])
    S2.append(s[n])
    
# Масксимальное и минимальное синглуярные числа
plt.plot(Ns, np.log10(S1), label = "max")
plt.plot(Ns, np.log10(S2), label = "min")
plt.legend()
plt.xlabel('N')
plt.ylabel('log S')
plt.grid(True)
plt.show()

# Зависимость числа обусловленности от размера сетки
plt.plot(Ns, [np.log10(S1[i]/S2[i]) for i in range(len(S2))], label = "Число обусловленности")
plt.legend()
plt.xlabel('N')
plt.ylabel('log cond')
plt.grid(True)
plt.show()

In [ ]:
n=256

In [ ]:
A = _A(n)
u, s, vh = np.linalg.svd(A, full_matrices=True)

# Сингулярные числа
plt.plot(list(range(A.shape[0])), np.log10(s), marker = ".")
plt.title("N = " + str(n))
plt.ylabel('log s')
plt.grid(True)
plt.show()

# 3

In [ ]:
noiseLevels = [0.0, 0.01,0.03, 0.05, 0.1]
A = _A(n)
b = _b(n)
x = np.linspace(0, ls, n+1)
qexs = np.zeros((n+1,1))
for i in range(n+1):
    qexs[i,0] =  qe(0 + i*(ls/n))

noise = np.transpose(np.random.rand(len(b))*2-1)    

for noiseLevel in noiseLevels:
    norm = np.linalg.norm(b)
    bD = b + (noise * norm * noiseLevel).reshape(b.shape)
    
    qsol = svd(A,bD)

    mse = ((np.transpose((qsol - qexs)) @ (qsol - qexs)) / n)[0,0]

    plt.plot(x, qsol, label = "Приближенное решение")
    plt.plot(x, qexs, label = "Точное решение")
    plt.legend()
    plt.title("N = " + str(n) + "; p = " + str(noiseLevel) + "; MSE = " + str(mse))
    plt.xlabel('N')
    plt.grid(True)
    plt.show()

# 4

In [ ]:
A = _A(n)
u, s, vh = np.linalg.svd(A, full_matrices=True)

d = s[0] - s[-1]

# Сингулярные числа матрица
#plt.plot(list(range(A.shape[0])), np.log(s), marker = ".")
plt.plot(list(range(A.shape[0])), np.log10(s))
plt.ylabel("log s")
plt.title("N = " + str(n))
plt.grid(True)

marker = []
for i in range(A.shape[0]-1):
    if(s[i] > 1.3 * s[i+1]):
        marker.append(i)

print(s[marker])

for i in marker:
    plt.plot(i, np.log10(s[i]), marker = ".")
        
plt.show()

Пример решения (уровень отсечки delta)

In [ ]:
noiseLevels = [0.05, 0.1]
delta = 1e-1
n=256
A = _A(n)
b = _b(n)
x = np.linspace(0, ls, n+1)
qexs = np.zeros((n+1,1))
for i in range(n+1):
    qexs[i,0] =  qe(0 + i*(ls/n))

noise = np.transpose(np.random.rand(len(b))*2-1)    

for noiseLevel in noiseLevels:

    norm = np.linalg.norm(b)
    bD = b + (noise * norm * noiseLevel).reshape(b.shape)
    
    qsol = csvd(A,bD, delta)

    mse = ((np.transpose((qsol - qexs)) @ (qsol - qexs)) / n)[0,0]
    plt.plot(x, qsol, label = "Приближенное решение")
    plt.plot(x, qexs, label = "Точное решение")
    plt.legend()
    plt.title("delta = " + str(delta) + "; p = " + str(noiseLevel) + "; MSE = " + str(mse))
    plt.xlabel('s')
    plt.grid(True)
    plt.show()

Поведение ошибки

In [ ]:
noiseLevels = [0.05, 0.1]
n=256
A = _A(n)
b = _b(n)
x = np.linspace(0, ls, n+1)
qexs = np.zeros((n+1,1))
for i in range(n+1):
    qexs[i,0] =  qe(0 + i*(ls/n))

noise = np.transpose(np.random.rand(len(b))*2-1)    

deltaRange = np.array(np.linspace(0, 1, 400) ** 10)

for noiseLevel in noiseLevels:
    err = []
    norm = np.linalg.norm(b)
    bD = b + (noise * norm * noiseLevel).reshape(b.shape)
    for delta in deltaRange:
        qsol = csvd(A,bD, delta)
        mse = ((np.transpose((qsol - qexs)) @ (qsol - qexs)) / n)[0,0]
        err.append(mse)
        
    plt.plot(np.log10(deltaRange), np.log10(err), marker=".")
    plt.title("p = " + str(noiseLevel))
    plt.ylabel("log MSE")
    plt.xlabel('log delta')

    plt.axvline(x = np.log10(1.18708503e+00), color = "r", ls="--")
    plt.axvline(x = np.log10(4.64857207e-02), color = "r", ls="--")
    plt.axvline(x = np.log10(1.45676089e-03), color = "r", ls="--")
    plt.axvline(x = np.log10(4.40233314e-05), color = "r", ls="--")
    plt.axvline(x = np.log10(1.31439876e-06), color = "r", ls="--")
    plt.axvline(x = np.log10(3.90401919e-08), color = "r", ls="--")
    plt.axvline(x = np.log10(1.15678965e-09), color = "r", ls="--")
    plt.axvline(x = np.log10(3.42433716e-11), color = "r", ls="--")
    plt.axvline(x = np.log10(1.01356484e-12), color = "r", ls="--")
    plt.axvline(x = np.log10(3.00197304e-14), color = "r", ls="--")
    plt.axvline(x = np.log10(9.32849550e-16), color = "r", ls="--")
    plt.axvline(x = np.log10(6.56841437e-17), color = "r", ls="--")

    plt.grid(True)
    plt.show()

# 5